In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import ipywidgets as widgets
from ipywidgets import interact



In [2]:
try:
    web_data = requests.get('https://finance.yahoo.com/quote/MSFT/sustainability?p=MSFT').text
    print("Success")
except:
    print("Could not get URL")
soup = BeautifulSoup(web_data, 'html.parser')
esg_score = soup.find('div', {'class':'Fz(36px) Fw(600) D(ib) Mend(5px)'})
print(esg_score)
#print(web_data)

Success
<div class="Fz(36px) Fw(600) D(ib) Mend(5px)" data-reactid="20">15</div>


In [3]:
datapoint = esg_score.text
datapoint

'15'

In [4]:
elements =[]
scores = soup.find_all('div', {'class': 'D(ib) Fz(23px) smartphone_Fz(22px) Fw(600)'})
for score in scores:
    elements.append(score.text)
elements

['0.5', '9.4', '5.1']

In [5]:
controversy_score = soup.find('div', {'class':'D(ib) Fz(36px) Fw(500)'})
controversy_datapoint = controversy_score.text


In [6]:
df = pd.DataFrame({'Total ESG Score': datapoint,
                   'Environment Score': elements[0],
                   'Social Score': elements[1],
                   'Governance Score': elements[2],
                   'Controversy Score': controversy_datapoint},
                  index=['MSFT'])
df

,Total ESG Score,Environment Score,Social Score,Governance Score,Controversy Score
MSFT,15,0.5,9.4,5.1,3


In [7]:
def get_tickers():
    wiki_page = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies').text
    sp_data = pd.read_html(wiki_page)
    ticker_df = sp_data[0]
    ticker_options = ticker_df['Symbol']
    return ticker_options


In [8]:
def web_scraper(ticker):
    elements = []
    web_data = requests.get('https://finance.yahoo.com/quote/'+ticker+'/sustainability?p='+ticker).text
    soup = BeautifulSoup(web_data, 'html.parser')
    esg_score = soup.find('div', {'class':'Fz(36px) Fw(600) D(ib) Mend(5px)'})
    datapoint = esg_score.text
    controversy_score = soup.find('div', {'class':'D(ib) Fz(36px) Fw(500)'})
    controversyz_datapoint = controversy_score.text
    scores = soup.find_all('div',{'class': 'D(ib) Fz(23px) smartphone_Fz(22px) Fw(600)'})
    for score in scores:
            elements.append(score.text)
            
    df = pd.DataFrame({'Total ESG Score': datapoint,
                   'Environment Score': elements[0],
                   'Social Score': elements[1],
                   'Governance Score': elements[2],
                   'Controversy Score': controversy_datapoint},
    index=[ticker])
    df = df.astype('float')	
    df['Controversy Assessment'] = df.apply(lambda x: level(x['Controversy Score']), axis=1)
    return df

In [9]:
def level(x):
    if x == 0.0:
        return 'No Controversy'
    elif x == 1.0:
        return 'Little Controversy'
    elif x == 2.0:
        return 'Relatively High Controversy'
    else: 
        return 'Severe Controversy'


In [10]:
@interact
def get_esg(ticker=widgets.Dropdown(description='Ticker',
                                    options=get_tickers(),
                                    value = 'F')):
    esg_data = web_scraper(ticker)
    return esg_data


interactive(children=(Dropdown(description='Ticker', index=198, options=('MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', …